<a href="https://colab.research.google.com/github/ShadowMonarch001/Advanced-News-Classsification-using-Big-Data-tools/blob/main/pyspark_newscategoryclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e6ab27beee00da0b7a1777579496407bc5b0f98b7e353f05d23664d4ee0e15c9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sc = SparkContext(master='local[2]')
spark = SparkSession.builder.appName("News Classification").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read dataset
df = spark.read.json("/content/drive/MyDrive/Classroom/News_Category_Dataset_v3.json")
df.show(5)

+--------------------+---------+----------+--------------------+--------------------+--------------------+
|             authors| category|      date|            headline|                link|   short_description|
+--------------------+---------+----------+--------------------+--------------------+--------------------+
|Carla K. Johnson, AP|U.S. NEWS|2022-09-23|Over 4 Million Am...|https://www.huffp...|Health experts sa...|
|      Mary Papenfuss|U.S. NEWS|2022-09-23|American Airlines...|https://www.huffp...|He was subdued by...|
|       Elyse Wanshel|   COMEDY|2022-09-23|23 Of The Funnies...|https://www.huffp...|"Until you have a...|
|    Caroline Bologna|PARENTING|2022-09-23|The Funniest Twee...|https://www.huffp...|"Accidentally put...|
|      Nina Golgowski|U.S. NEWS|2022-09-22|Woman Who Called ...|https://www.huffp...|Amy Cooper accuse...|
+--------------------+---------+----------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df = df.select("headline","short_description","category")
df.show(5)

+--------------------+--------------------+---------+
|            headline|   short_description| category|
+--------------------+--------------------+---------+
|Over 4 Million Am...|Health experts sa...|U.S. NEWS|
|American Airlines...|He was subdued by...|U.S. NEWS|
|23 Of The Funnies...|"Until you have a...|   COMEDY|
|The Funniest Twee...|"Accidentally put...|PARENTING|
|Woman Who Called ...|Amy Cooper accuse...|U.S. NEWS|
+--------------------+--------------------+---------+
only showing top 5 rows



In [ ]:
df.count()

209527

In [ ]:
df.groupby('category').count().sort('count', ascending=False).show()

+--------------+-----+
|      category|count|
+--------------+-----+
|      POLITICS|35602|
|      WELLNESS|17945|
| ENTERTAINMENT|17362|
|        TRAVEL| 9900|
|STYLE & BEAUTY| 9814|
|     PARENTING| 8791|
|HEALTHY LIVING| 6694|
|  QUEER VOICES| 6347|
|  FOOD & DRINK| 6340|
|      BUSINESS| 5992|
|        COMEDY| 5400|
|        SPORTS| 5077|
|  BLACK VOICES| 4583|
| HOME & LIVING| 4320|
|       PARENTS| 3955|
| THE WORLDPOST| 3664|
|      WEDDINGS| 3653|
|         WOMEN| 3572|
|         CRIME| 3562|
|        IMPACT| 3484|
+--------------+-----+
only showing top 20 rows



In [ ]:
# check for missing value
print("Headline ", df.toPandas()['headline'].isnull().sum())
print("short_description ", df.toPandas()['short_description'].isnull().sum())
print("category ", df.toPandas()['category'].isnull().sum())

Headline  0
short_description  0
category  0


In [ ]:
from pyspark.sql import functions as sf
df = df.withColumn('description',
                    sf.concat(sf.col('headline'),sf.lit(' '), sf.col('short_description')))

# selecting only 2 columns
df = df.select("description","category")
df.show()

+--------------------+--------------+
|         description|      category|
+--------------------+--------------+
|Over 4 Million Am...|     U.S. NEWS|
|American Airlines...|     U.S. NEWS|
|23 Of The Funnies...|        COMEDY|
|The Funniest Twee...|     PARENTING|
|Woman Who Called ...|     U.S. NEWS|
|Cleaner Was Dead ...|     U.S. NEWS|
|Reporter Gets Ado...|     U.S. NEWS|
|Puerto Ricans Des...|    WORLD NEWS|
|How A New Documen...|CULTURE & ARTS|
|Biden At UN To Ca...|    WORLD NEWS|
|World Cup Captain...|    WORLD NEWS|
|Man Sets Himself ...|    WORLD NEWS|
|Fiona Threatens T...|    WORLD NEWS|
|Twitch Bans Gambl...|          TECH|
|Virginia Thomas A...|     U.S. NEWS|
|Russian Cosmonaut...|    WORLD NEWS|
|'Reboot' Is A Cle...|CULTURE & ARTS|
|Maury Wills, Base...|        SPORTS|
|4 Russian-Control...|    WORLD NEWS|
|Fiona Barrels Tow...|    WORLD NEWS|
+--------------------+--------------+
only showing top 20 rows



In [ ]:
# test_df = spark.createDataFrame([("this is some","this is some other")],['1','2'])
# test_df = test_df.withColumn('description',
#                     sf.concat(sf.col('1'),sf.lit(' '), sf.col('2')))
# test_df.show(truncate=False)

### Feature Extraction

build features
+ count vectorizer
+ tfIDF
+ wordEmbeddings
+ hashingTF
+ etc...

We have 2 things in Pipeline stages
- Transformer
- Estimator

**Transformer** (Data to Data)

Function that takes data and fit, transform them into augmented data or features
i.e Extractors, Vectorizer, Scalers (Tokenizer, StopwordRemover, CountVectorizer, IDF)

**Estimator** (Data to model)

Function that takes data as input and fit the data and produces a model we can use to predict
i.e LogisticRegression

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [ ]:
df.columns

['description', 'category']

In [ ]:
import pyspark.sql.functions as f
# df = dfRawData.where(f.col("X").isin(["CB", "CI", "CR"]))
df = df.where(f.col('description') != " ")
df.show()

+--------------------+--------------+
|         description|      category|
+--------------------+--------------+
|Over 4 Million Am...|     U.S. NEWS|
|American Airlines...|     U.S. NEWS|
|23 Of The Funnies...|        COMEDY|
|The Funniest Twee...|     PARENTING|
|Woman Who Called ...|     U.S. NEWS|
|Cleaner Was Dead ...|     U.S. NEWS|
|Reporter Gets Ado...|     U.S. NEWS|
|Puerto Ricans Des...|    WORLD NEWS|
|How A New Documen...|CULTURE & ARTS|
|Biden At UN To Ca...|    WORLD NEWS|
|World Cup Captain...|    WORLD NEWS|
|Man Sets Himself ...|    WORLD NEWS|
|Fiona Threatens T...|    WORLD NEWS|
|Twitch Bans Gambl...|          TECH|
|Virginia Thomas A...|     U.S. NEWS|
|Russian Cosmonaut...|    WORLD NEWS|
|'Reboot' Is A Cle...|CULTURE & ARTS|
|Maury Wills, Base...|        SPORTS|
|4 Russian-Control...|    WORLD NEWS|
|Fiona Barrels Tow...|    WORLD NEWS|
+--------------------+--------------+
only showing top 20 rows



In [ ]:
# stages for pipeline
tokenizer = Tokenizer(inputCol='description',outputCol='mytokens')
stopwordRemover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [ ]:
# encode the label feature (category)
labelEncoder = StringIndexer(inputCol='category',outputCol='label').fit(df)
labelEncoder.transform(df).show(5)

+--------------------+---------+-----+
|         description| category|label|
+--------------------+---------+-----+
|Over 4 Million Am...|U.S. NEWS| 36.0|
|American Airlines...|U.S. NEWS| 36.0|
|23 Of The Funnies...|   COMEDY| 10.0|
|The Funniest Twee...|PARENTING|  5.0|
|Woman Who Called ...|U.S. NEWS| 36.0|
+--------------------+---------+-----+
only showing top 5 rows



In [ ]:
labelEncoder.labels

['POLITICS',
 'WELLNESS',
 'ENTERTAINMENT',
 'TRAVEL',
 'STYLE & BEAUTY',
 'PARENTING',
 'HEALTHY LIVING',
 'QUEER VOICES',
 'FOOD & DRINK',
 'BUSINESS',
 'COMEDY',
 'SPORTS',
 'BLACK VOICES',
 'HOME & LIVING',
 'PARENTS',
 'THE WORLDPOST',
 'WEDDINGS',
 'WOMEN',
 'CRIME',
 'IMPACT',
 'DIVORCE',
 'WORLD NEWS',
 'MEDIA',
 'WEIRD NEWS',
 'GREEN',
 'WORLDPOST',
 'RELIGION',
 'STYLE',
 'SCIENCE',
 'TECH',
 'TASTE',
 'MONEY',
 'ARTS',
 'ENVIRONMENT',
 'FIFTY',
 'GOOD NEWS',
 'U.S. NEWS',
 'ARTS & CULTURE',
 'COLLEGE',
 'LATINO VOICES',
 'CULTURE & ARTS',
 'EDUCATION']

In [ ]:
# labelEncoder.labels
label_dict =  {
 'POLITICS':0.0,
 'ENTERTAINMENT':1.0,
 'WORLD NEWS':2.0,
 'QUEER VOICES':3.0,
 'COMEDY':4.0,
 'BLACK VOICES':5.0,
 'SPORTS':6.0,
 'MEDIA':7.0,
 'WOMEN':8.0,
 'WEIRD NEWS':9.0,
 'CRIME':10.0,
 'BUSINESS':11.0,
 'LATINO VOICES':12.0,
 'IMPACT':13.0,
 'RELIGION':14.0,
 'TRAVEL':15.0,
 'STYLE':16.0,
 'GREEN':17.0,
 'PARENTS':18.0,
 'TECH':20.0,
 'HEALTHY LIVING':21.0,
 'SCIENCE':22.0,
 'EDUCATION':23.0,
 'TASTE':24.0,
 'ARTS & CULTURE':25.0,
 'COLLEGE':26.0
}

In [ ]:
df = labelEncoder.transform(df)
df.show(5)

+--------------------+---------+-----+
|         description| category|label|
+--------------------+---------+-----+
|Over 4 Million Am...|U.S. NEWS| 36.0|
|American Airlines...|U.S. NEWS| 36.0|
|23 Of The Funnies...|   COMEDY| 10.0|
|The Funniest Twee...|PARENTING|  5.0|
|Woman Who Called ...|U.S. NEWS| 36.0|
+--------------------+---------+-----+
only showing top 5 rows



In [ ]:
# split dataset
(train_df,test_df) = df.randomSplit((0.7,0.3),seed=42)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [ ]:
# building the pipeline
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(
    stages = [tokenizer, stopwordRemover, vectorizer,idf,lr]
    )

In [ ]:
pipeline.stages

Param(parent='Pipeline_e7edb9f25b7a', name='stages', doc='a list of pipeline stages')

In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [ ]:
lr_model = pipeline.fit(train_df)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.10/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python

Py4JError: py4j.reflection does not exist in the JVM

In [ ]:
lr_model

In [ ]:
# get prediction on test data
predictions = lr_model.transform(test_df)

In [ ]:
predictions.columns

In [ ]:
predictions.select('description','rawPrediction', 'probability','category','label','prediction').show(10)

### model evaluation
+ Accuracy
+ Precision
+ F1Score
+ etc

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                             labelCol='label')

evaluator.evaluate(predictions)*100

#### making prediction on one sample
+ sample as df
+ apply pipeline

In [ ]:
from pyspark.sql.types import StringType

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Test11") \
    .getOrCreate()


In [ ]:
test1 = spark.createDataFrame([
    ("Who will be Uttarakhand CM? BJP leaders hold meet today; Trivendra Rawat says have many options",),
    ("AAP nominations for Rajya Sabha polls a 'betrayal' of Punjab: Navjot Singh Sidhu",),
    ("Argentina Team Pulls Off Stunning Upset in Championship Match of Football",),
    ("New Netflix movie Saw Series Breaks Records with Jaw-Dropping Premiere Weekend",)
], ['description'])

# Collect the data


In [ ]:
prediction_test = lr_model.transform(test1)
print(prediction_test)

In [ ]:
prediction_test.columns

In [ ]:
prediction_test.select('description','rawPrediction','probability','prediction').show()

In [ ]:
label_dict

In [ ]:
lr_model.save("mypipeline")

In [ ]:
  x!zip -r /content/file.zip /content/mypipeline

In [ ]:
 +